## OBJECTIVE

We present here two methods of data augmentation for working with portuguese texts, that could be used with any other language. If using english, for example, the idea is to translate the text to some other language and then back to english.

The goal is to increment the dataset with artificially created texts to create new examples of the least sampled class. This should be done only in the training dataset.

In [1]:
import pandas as pd
from googletrans import Translator

In [2]:
MOTHER_LANGUAGE = "pt"
TRANSLATED_LANGUAGE = "en"

In [3]:
def create_texts_from_translation(df, class_col="class", text_col="text"):
    """
        Input: DataFrame with a binary 'class' column and a 'text' column
        Output: An augmented DataFrame
    """
    class_distribution = df[class_col].mean()
    if class_distribution == 0.5:
        raise Exception("Data augmentation not necessary for balanced class")
    augmented_class = 1 if class_distribution < 0.5 else 0
    number_new_texts = len(df[df[class_col] != augmented_class]) - len(df[df[class_col] == augmented_class])
    aux = number_new_texts
    new_rows = []
    translator = Translator()
    while aux > 0:
        for row in df.to_dict("records"):
            new_rows.append({
                text_col:translator.translate(translator.translate(row[text_col], src=MOTHER_LANGUAGE, dest=TRANSLATED_LANGUAGE).text, src=TRANSLATED_LANGUAGE, dest=MOTHER_LANGUAGE).text,
                class_col:augmented_class,
            })
            aux -= 1
            if aux == 0:
                break
    df_aux = pd.DataFrame(new_rows)
    return pd.concat([df, df_aux])

### EXAMPLE

In [4]:
example_df = pd.DataFrame([
    {"text":"Esta lei é inconstitucional","class":0},
    {"text":"A inconstitucionalidade é flagrante","class":0},
    {"text":"Esta lei desrespeita a constituição federal","class":0},
    {"text":"A lei inconstitucional não pode ser aplicada","class":0},
    {"text":"Esta lei foi declarada inconstitucional","class":0},
    {"text":"O dispositivo legislativo foi revogado","class":0},
    {"text":"O juiz optou por não aplicar a lei","class":0},
    {"text":"A lei é constitucional","class":1},
    {"text":"O juiz optou por aplicar a lei","class":1},
    {"text":"A lei foi declarada constitucional pelo tribunal","class":1},
])

In [5]:
example_df_augmented = create_texts_from_translation(example_df)

In [6]:
for row in example_df_augmented.to_dict("records"):
    print(row)

{'text': 'Esta lei é inconstitucional', 'class': 0}
{'text': 'A inconstitucionalidade é flagrante', 'class': 0}
{'text': 'Esta lei desrespeita a constituição federal', 'class': 0}
{'text': 'A lei inconstitucional não pode ser aplicada', 'class': 0}
{'text': 'Esta lei foi declarada inconstitucional', 'class': 0}
{'text': 'O dispositivo legislativo foi revogado', 'class': 0}
{'text': 'O juiz optou por não aplicar a lei', 'class': 0}
{'text': 'A lei é constitucional', 'class': 1}
{'text': 'O juiz optou por aplicar a lei', 'class': 1}
{'text': 'A lei foi declarada constitucional pelo tribunal', 'class': 1}
{'text': 'Esta lei é inconstitucional', 'class': 1}
{'text': 'A inconstitucionalidade é flagrante', 'class': 1}
{'text': 'Esta lei desrespeita a Constituição Federal', 'class': 1}
{'text': 'A lei inconstitucional não pode ser aplicada', 'class': 1}
